In [1]:
import sqlite3
import pandas as pd

In [2]:
# --- Config ---
CSV_FILE = r"D:\CNTT_Shared\OOP\OOP_Summer_2025_Project\src\main\resources\db\Books.csv"  # path to your CSV file
DB_FILE = r"D:\CNTT_Shared\OOP\OOP_Summer_2025_Project\src\main\resources\db\library_new.db"  # your SQLite file

In [3]:
# --- Load CSV ---
df = pd.read_csv(CSV_FILE)

# Normalize column names (in case of weird spacing)
df.columns = [col.strip() for col in df.columns]

# --- Connect to SQLite ---
conn = sqlite3.connect(DB_FILE)
cursor = conn.cursor()

# --- Create table if not exists ---
cursor.execute("""
CREATE TABLE IF NOT EXISTS books (
    bookId TEXT PRIMARY KEY,
    isbn TEXT,
    book_title TEXT NOT NULL,
    book_author TEXT NOT NULL,
    year_of_publication INTEGER,
    publisher TEXT,
    borrowCount INTEGER DEFAULT 0,
    image_url_s TEXT,
    image_url_m TEXT,
    image_url_l TEXT
)
""")

# --- Insert Data ---
for _, row in df.iterrows():
    bookId = f"{row['ISBN']}-{row['Year-Of-Publication']}"  # composite key
    
    cursor.execute("""
        INSERT OR IGNORE INTO books (
            bookId, isbn, book_title, book_author, year_of_publication, 
            publisher, image_url_s, image_url_m, image_url_l
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, (
        bookId,
        row['ISBN'],
        row['Book-Title'],
        row['Book-Author'],
        row['Year-Of-Publication'],
        row['Publisher'],
        row['Image-URL-S'],
        row['Image-URL-M'],
        row['Image-URL-L']
    ))

# --- Commit & Close ---
conn.commit()
conn.close()

print("✅ Books imported successfully!")

C:\Users\HLC\AppData\Local\Temp\ipykernel_19952\3112567127.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(CSV_FILE)


✅ Books imported successfully!


In [7]:
def query_rows(query, params=None):
    """
    Run a query against SQLite and return results as a pandas DataFrame.
    """
    conn = sqlite3.connect(DB_FILE)
    try:
        if params:
            df = pd.read_sql_query(query, conn, params=params)
        else:
            df = pd.read_sql_query(query, conn)
    finally:
        conn.close()
    return df


sql = f"SELECT * FROM books LIMIT 10"
result = query_rows(sql)

print(result)


            bookId        isbn  \
0  0195153448-2002  0195153448   
1  0002005018-2001  0002005018   
2  0060973129-1991  0060973129   
3  0374157065-1999  0374157065   
4  0393045218-1999  0393045218   
5  0399135782-1991  0399135782   
6  0425176428-2000  0425176428   
7  0671870432-1993  0671870432   
8  0679425608-1996  0679425608   
9  074322678X-2002  074322678X   

                                          book_title           book_author  \
0                                Classical Mythology    Mark P. O. Morford   
1                                       Clara Callan  Richard Bruce Wright   
2                               Decision in Normandy          Carlo D'Este   
3  Flu: The Story of the Great Influenza Pandemic...      Gina Bari Kolata   
4                             The Mummies of Urumchi       E. J. W. Barber   
5                             The Kitchen God's Wife               Amy Tan   
6  What If?: The World's Foremost Military Histor...         Robert Cowley   
7

In [8]:
conn.close()